## 十六. 向量存储索引（Vectorstore-Index）

本例将使用 Vectorstore-Index 来简化 QA Chain 流程。

使用 QA Chain 实现基于文档问答功能，至少包括五个前置步骤：

1. 文档加载（Document Loading）：文档加载器把文档加载为 LangChain 能够读取的形式。有不同类型的加载器来加载不同数据源的数据，如CSVLoader、PyPDFLoader、Docx2txtLoader、TextLoader等。
2. 文本分割（Splitting）：文本分割器把 Documents 切分为指定大小的分割，分割后的文本称为“文档块”或者“文档片”。（本次忽略）
3. 向量存储（Vector Storage）：将上一步中分割好的“文档块”以“嵌入”（Embedding）的形式存储到向量数据库（Vector DB）中，形成一个个的“嵌入片”。
4. 检索（Retrieval）：应用程序从存储中检索分割后的文档（例如通过比较余弦相似度，找到与输入问题类似的嵌入片）。
5. 构建QA链 (Chain)：组装 LLM 和 Retriever
6. 输出（Output）：把问题和相似的嵌入片（文本形式）都放到提示传递给 Chain，让大语言模型生成答案。

### 完整代码如下：

In [ ]:
from langchain.chains import RetrievalQA 
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import CSVLoader 
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.chat_models import ChatOpenAI


#1.加载文档,并进行文档切割
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
docs = loader.load()
 
#2.创建embeddings
embeddings = OpenAIEmbeddings()
 
#3.创建向量数据库
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)
 
#4.创建检索器
retriever = db.as_retriever()
 
#5.创建RetrievalQA
qa_stuff = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(temperature = 0.0), 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

# 6. 向文档提出问题
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."
 
response = qa_stuff.run(query)
 
display(Markdown(response))

In [ ]:
# 基于DocArray的内存方式的向量存储
# !pip show docarray

!pip install "langchain[docarray]"

上面的步骤有点繁琐，我们可以进一步通过 LangChain 的 VectorstoreIndex 来简化

In [1]:
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.indexes import VectorstoreIndexCreator
from IPython.display import display, Markdown


| 工具                     | 用途                         |
| ---------------------- | -------------------------- |
| CSVLoader              | 加载专用数据(如CSV)，用来与模型结合使用     |
| DocArrayInMemorySearch | 基于DocArray的内存方式的向量存储，不需要连接到任何类型的外部数据库 |
| VectorstoreIndexCreator | 用于创建向量存储索引，封装文档加载、分块、嵌入、存储、召回等功能 |

In [2]:
# 步骤1：初始化CSV加载器，为其指定一个CSV文件的路径

file = './OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

In [3]:
# 步骤2：创建一个内存方式的向量存储，传入上一步创建的️️加载器。

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [4]:
# 步骤3：定义查询内容，并使用index.query生成响应
query ="Please list all your shirts with sun protection in a table in markdown and summarize each one."
response = index.query(query)


可以看到，使用 VectorstoreIndex 三步就可以实现一个完整的基于文档问答的功能了。

In [15]:
# 展示查询结果的表格以及摘要
display(Markdown(response))



| Shirt ID | Name | Sun Protection |
| -------- | ---- | -------------- |
| 618 | Men's Tropical Plaid Short-Sleeve Shirt | UPF 50+ rated, blocks 98% of the sun's harmful rays |
| 374 | Men's Plaid Tropic Shirt, Short-Sleeve | UPF 50+ rated, blocks 98% of the sun's harmful rays |
| 255 | Sun Shield Shirt by | UPF 50+ rated, blocks 98% of the sun's harmful rays |
| 535 | Men's TropicVibe Shirt, Short-Sleeve | UPF 50+ rated, blocks 98% of the sun's harmful rays |

All of the shirts listed provide UPF 50+ rated sun protection, blocking 98% of the sun's harmful rays. The Men's Tropical Plaid Short-Sleeve Shirt is made of 100% polyester and is wrinkle-resistant. The Men's Plaid Tropic Shirt, Short-Sleeve is made of 52% polyester and 48% nylon and is machine washable and dryable. The Sun Shield Shirt by is made of 78% nylon and 22% Lycra Xtra Life fiber and is handwashable and line

In [7]:
index.query_with_sources(query)

{'question': 'Please list all your shirts with sun protection in a table in markdown and summarize each one.',
 'answer': " \n\n| Shirt ID | Name | Description |\n| -------- | ---- | ----------- |\n| 618 | Men's Tropical Plaid Short-Sleeve Shirt | Our lightest hot-weather shirt is rated UPF 50+ for superior protection from the sun's UV rays. With a traditional fit that is relaxed through the chest, sleeve, and waist, this fabric is made of 100% polyester and is wrinkle-resistant. With front and back cape venting that lets in cool breezes and two front bellows pockets, this shirt is imported and provides the highest rated sun protection possible. Sun Protection That Won't Wear Off. Our high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun's harmful rays. |\n| 374 | Men's Plaid Tropic Shirt, Short-Sleeve | Our Ultracomfortable sun protection is rated to UPF 50+, helping you stay cool and dry. Originally designed for fishing, this lightest hot-weather shirt offe

从 VectorstoreIndexCreator 返回的是 VectorStoreIndexWrapper，它提供了这些 query 和 query_with_sources 功能。如果您想直接访问 vectorstore，也可像下面这样：

In [10]:
db = index.vectorstore
db

In [11]:
query = "Please suggest a shirt with sunblocking"
docs = db.similarity_search(query)

docs

[Document(page_content='shirt id: 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \n\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\n\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\n\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\n\nSun Protection That Won\'t Wear Off\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.', metadata={'source': './OutdoorClothingCatalog_1000.csv', 'row': 255}),
 Document(page_content="shirt id: 374\nname: Men's Plaid Tropic Shirt, Short-Sleeve\ndescription: Our Ultracomfortable sun protecti

也可以访问 VectorstoreRetriever：

In [17]:
retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['DocArrayInMemorySearch'], vectorstore=<langchain.vectorstores.docarray.in_memory.DocArrayInMemorySearch object at 0x7f937841d630>)

In [18]:
retriever.get_relevant_documents(query)

[Document(page_content='shirt id: 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \n\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\n\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\n\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\n\nSun Protection That Won\'t Wear Off\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.', metadata={'source': './OutdoorClothingCatalog_1000.csv', 'row': 255}),
 Document(page_content="shirt id: 374\nname: Men's Plaid Tropic Shirt, Short-Sleeve\ndescription: Our Ultracomfortable sun protecti

可以看到：


VectorstoreIndexCreator 仅仅是所有逻辑的一个简单封装。它涵盖了文本分割器的使用、使用哪种嵌入以及使用哪种 vectorstore。

如果您想进行配置方面的修改，可以按以下方式进行：

In [ ]:
index_creator = VectorstoreIndexCreator(
    vectorstore_cls=Chroma, 
    embedding=OpenAIEmbeddings(),
    text_splitter=CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
)

## 总结：Index 与  RetrievalQA

用index.query()这个方法只用一行代码也可以实现文档问答的功能，也许有读者会问，既然index.query()可以只用一行代码就完成了文档问答功能又何必要舍近求远搞一个RetrievalQA这样的对象来实现，并且增加很多繁琐的步骤(有5个步骤)来实现同样的效果呢？

Langchain框架的作者Harrison Chase在课件视频中是这么解释的，通过index来进行文档问答，只需一行代码，但是这其中其实隐藏了很多的实现细节，如果我们使用的是RetrievalQA对象来实现文档问答功能，那么我们就可以了解其中的细节比如Embeddings，向量数据库等内容，反正各有各的好处吧。


## 参考链接

https://learn.deeplearning.ai/langchain/lesson/5/question-and-answer

https://python.langchain.com/docs/modules/data_connection/vectorstores/integrations/docarray_in_memory